In [1]:
# Start writing code here...
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import igraph

In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor rele

This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [2]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
os.listdir("../data")

['.DS_Store',
 'description.txt',
 'input_wagon.xlsx',
 'metrics.csv.gz',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'output_wagon.xlsx',
 'sources.csv.gz',
 'standplaces.csv.gz',
 'wagon.zip',
 'wagon_mode_compat.csv.gz']

In [5]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz") # +
sources = pd.read_csv("../data/sources.csv.gz") # +
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz") #+

In [6]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.abstractions import *

In [7]:
roads_graph, name2vertex = get_roads_graph(metrics, debug=False)

In [8]:
roads_graph.es[0]['info'], roads_graph.es[0]['distance'], roads_graph.es[0]['time']

([(100, 104), (200, 138), (305, 355)], 9, 5)

In [9]:
roads_graph.es['weight'] = roads_graph.es['distance']

In [10]:
# dijkstra_paths = roads_graph.shortest_paths_dijkstra(weights="weight")

In [11]:
# np.min(dijkstra_paths), np.max(dijkstra_paths), np.mean(dijkstra_paths), np.median(dijkstra_paths)
sources[sources.Station==190205]

,Unnamed: 0,Station,Date,Units,Stand,WagonModel


In [12]:
gr = init_stations(sources, name2vertex)


In [13]:
190205
gr[0].vs[0]['info']

{'2070': [(1, 50), (1, 50)],
 '2071': [(1, 50)],
 '2135': [(1, 50)],
 '2062': [(1, 50)],
 '2155': [(1, 50)]}

In [14]:
gr, wagons = get_wagon_graph(wagon_mode_compat)

In [15]:
def create_order_graph(orders, name2vertex):
    columns = [
        "OrderNum",
        "Start",
        "Finish",
        #"StartDate",
        "Dur",
        "MinUnit",
        "MaxUnit",
        "Tariff",
        "NeedWagonModel",
        "ShortagePenalty"
    ]
    vertices = np.unique(np.concatenate([
        orders.Start.values,
        orders.Finish.values
    ]))
    for v in vertices:
        v = str(v)
        if v not in name2vertex:
            index = len(name2vertex)
            name2vertex[v] = index
    graph = Graph()
    graph.add_vertices(len(name2vertex))
    edges = []
    for start, finish in orders[["Start", "Finish"]].values:
        start = str(start)
        finish = str(finish)
        u = name2vertex[start]
        v = name2vertex[finish]
        edges.append((u, v))
    # print(edges)
    graph.add_edges(edges)
    graph['order_num'] = orders["OrderNum"].values
    graph['dur'] = orders["Dur"].values
    graph['min_unit'] = orders["MinUnit"].values
    graph['max_unit'] = orders["MaxUnit"].values
    graph['tariff'] = orders["Tariff"].apply(lambda xs: [int(x) for x in xs.split(":")]).values
    graph["wagon_model"] = orders["NeedWagonModel"].values
    graph["penalty"] = orders["ShortagePenalty"].values
    return graph
    
def build_orders(orders, name2vertex):
    columns = [
        "OrderNum",
        "Start",
        "Finish",
        "StartDate",
        "Dur",
        "MinUnit",
        "MaxUnit",
        "Tariff",
        "NeedWagonModel",
        "ShortagePenalty"
    ]
    graphs = dict()
    for start_date, rows in orders7[columns].groupby("StartDate"):
        graph = create_order_graph(rows, name2vertex)
        graphs[start_date] = graph
    # return start_date, rows
    return graphs

In [16]:
graphs = build_orders(orders7, name2vertex)

In [17]:
env = WorldModelEnv(
    orders=orders7,
    wagon_mode_compat=wagon_mode_compat,
    sources=sources, metrics=metrics)

In [18]:
gr = env.init_world_graph()

In [19]:
gr.es[0]['route'].arrivals[1] = (1,2,3)

In [20]:
gr.es[2]['route'].arrivals

{}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ba67d69-ce72-437b-8bf4-6d3c9b906c4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

In [22]:
env.station2vertex['104202']

1590

In [29]:
(metrics.From == 104202).any()

False

In [32]:
metrics[(metrics.From == 104202) | (metrics.To==104202)]

,Unnamed: 0,From,To,Group,Distance,Time,PriceUnit


In [33]:
metrics

,Unnamed: 0,From,To,Group,Distance,Time,PriceUnit
0,1,10002,10303,100,9,5,104
1,2,10002,10303,200,9,5,138
2,3,10002,10303,305,9,5,355
3,4,10002,10905,100,60,5,685
4,5,10002,10905,200,60,5,914
...,...,...,...,...,...,...,...
939399,939400,998100,980605,304,2243,16,55310
939400,939401,998100,984502,304,2372,16,59330
939401,939402,998100,988306,304,2126,15,53478
939402,939403,998100,988306,313,2126,15,50353
